In [1]:
#テスト
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [2]:
train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../input/titanic/train.csv'

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.describe()

In [ ]:
train.drop(["PassengerId", "Name", "Ticket"], axis=1, inplace=True)
train.head()

In [ ]:
test.drop(["PassengerId", "Name", "Ticket"], axis=1, inplace=True)
test.head()

In [2]:
label = train["Survived"]
train.drop(["Survived"], axis=1, inplace=True)

NameError: name 'train' is not defined

In [3]:
train.shape
train.dtypes

NameError: name 'train' is not defined

In [10]:
cat_feat = ["Pclass", "Sex", "Cabin", "Embarked"]
for i in cat_feat :
    train[i] = train[i].astype("category")
    test[i] = test[i].astype("category")

In [11]:
train.shape
train.dtypes

Pclass      category
Sex         category
Age          float64
SibSp          int64
Parch          int64
Fare         float64
Cabin       category
Embarked    category
dtype: object

In [12]:
X_train, X_valid, y_train, y_valid = train_test_split(train, label)

In [13]:
lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=cat_feat)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=cat_feat)

params = {
    "objective" : "binary",
    "metric" : "binary_logloss",
    "early_stopping_round" : 10
}

In [14]:
model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_eval])

[LightGBM] [Info] Number of positive: 264, number of negative: 404
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008321 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 205
[LightGBM] [Info] Number of data points in the train set: 668, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.395210 -> initscore=-0.425466
[LightGBM] [Info] Start training from score -0.425466
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	training's binary_logloss: 0.623907	valid_1's binary_logloss: 0.613454
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	training's binary_logloss: 0.585643	valid_1's binary_logloss: 0.583028
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	trainin

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


In [15]:
val_pred = model.predict(X_valid, num_iteration=model.best_iteration)
val_pred

array([0.43674035, 0.72414542, 0.50099581, 0.27496062, 0.06736322,
       0.87478528, 0.28055904, 0.39190555, 0.07307171, 0.22523764,
       0.09565072, 0.19626125, 0.29877477, 0.1760157 , 0.16526904,
       0.1898221 , 0.24135964, 0.29877477, 0.50099581, 0.72414542,
       0.5657959 , 0.74704714, 0.33126596, 0.25769792, 0.23880455,
       0.80492865, 0.18338437, 0.44184911, 0.1277235 , 0.22568647,
       0.22084246, 0.21926572, 0.17936653, 0.0984209 , 0.22148818,
       0.36761872, 0.55736612, 0.27317547, 0.37691844, 0.07307171,
       0.74704714, 0.85816109, 0.26096738, 0.24636486, 0.07895609,
       0.39190555, 0.44841784, 0.18222185, 0.14721567, 0.14451304,
       0.17259894, 0.886881  , 0.18176565, 0.0619277 , 0.90694671,
       0.15041292, 0.74704714, 0.27317547, 0.35128766, 0.16660059,
       0.39190555, 0.16753494, 0.11896372, 0.23980436, 0.88059413,
       0.11190707, 0.32540456, 0.5657959 , 0.87237029, 0.88802046,
       0.06093598, 0.27186061, 0.14006588, 0.08837079, 0.08621

In [16]:
val_pred = np.where(val_pred < 0.5, 0, 1)
acc = accuracy_score(y_valid, val_pred)
acc

0.8385650224215246

In [17]:
test_pred = model.predict(test, num_iteration=model.best_iteration)
test_pred = np.where(test_pred < 0.5, 0, 1)
test_pred

array([0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [18]:
sub = pd.read_csv("../input/titanic/gender_submission.csv")

In [19]:
sub.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [20]:
sub["Survived"] = test_pred
sub

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [21]:
sub.to_csv("submission.csv", index=False)